In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
j = pd.read_csv('./Kaggle Draft/nfl_draft_prospects.csv')
draft = pd.read_csv('./Kaggle Draft/nfl_draft_profiles.csv')

In [4]:
scout = draft.merge(j,how='left',on='player_id')

In [5]:
scout[scout.draft_year >= 1995]

,player_id,guid_x,alt_player_id,player_name_x,position_x,pos_abbr_x,weight_x,height_x,player_image_x,link_x,...,team,team_abbr,team_logo_espn,guid_y,weight_y,height_y,pos_rk_y,ovr_rk_y,grade_y,player_image_y
5129,764,NaN,NaN,Ki-Jana Carter,Running Back,RB,0,0.000,NaN,http://insider.espn.com/nfl/draft/player/_/id/764,...,Cincinnati Bengals,CIN,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5130,4517,NaN,NaN,Tony Boselli,Offensive Tackle,OT,0,0.000,NaN,http://insider.espn.com/nfl/draft/player/_/id/...,...,Jacksonville Jaguars,JAX,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5131,4518,NaN,NaN,Steve McNair,Quarterback,QB,0,0.000,NaN,http://insider.espn.com/nfl/draft/player/_/id/...,...,Houston Oilers,HOU,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5132,4519,NaN,NaN,Michael Westbrook,Wide Receiver,WR,0,0.000,NaN,http://insider.espn.com/nfl/draft/player/_/id/...,...,Washington Redskins,WSH,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5133,4520,NaN,NaN,Kerry Collins,Quarterback,QB,0,0.000,NaN,http://insider.espn.com/nfl/draft/player/_/id/...,...,Carolina Panthers,CAR,https://a.espncdn.com/i/teamlogos/nfl/500/scor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12900,105466,841ed58127d19410cc2849ea8f3e1070,3121655.0,Justus Reed,Defensive End,DE,253,75.000,NaN,http://insider.espn.com/nfl/draft/player/_/id/...,...,NaN,NaN,NaN,841ed58127d19410cc2849ea8f3e1070,253.0,75.000,31.0,344.0,30.0,NaN
12901,105467,151d962cf4bf1862bcef120a11880e57,4044431.0,K.J. Costello,Quarterback,QB,227,76.625,NaN,http://insider.espn.com/nfl/draft/player/_/id/...,...,NaN,NaN,NaN,151d962cf4bf1862bcef120a11880e57,227.0,76.625,15.0,345.0,30.0,NaN
12902,105468,d12830501cce353220df8c6bd2b050fc,4240599.0,Donovan Stiner,Safety,S,205,73.500,NaN,http://insider.espn.com/nfl/draft/player/_/id/...,...,NaN,NaN,NaN,d12830501cce353220df8c6bd2b050fc,205.0,73.500,29.0,346.0,30.0,NaN
12903,105478,8d1939ce8d84d85651dcff926bebc08b,4369247.0,Mac McCain III,Cornerback,CB,186,71.000,https://a.espncdn.com/i/headshots/nfldraft/pla...,http://insider.espn.com/nfl/draft/player/_/id/...,...,NaN,NaN,NaN,8d1939ce8d84d85651dcff926bebc08b,186.0,71.000,44.0,348.0,30.0,https://a.espncdn.com/i/headshots/nfldraft/pla...


In [6]:
scout = scout[['player_name_x','pos_abbr_x','weight_x','height_x','school_x','pos_rk_x','ovr_rk_x','grade_x','text1','text2','text3','text4','draft_year','overall','round','team_abbr']]

In [7]:
scout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12905 entries, 0 to 12904
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  12905 non-null  object 
 1   pos_abbr_x     12905 non-null  object 
 2   weight_x       12905 non-null  int64  
 3   height_x       12905 non-null  float64
 4   school_x       12905 non-null  object 
 5   pos_rk_x       5761 non-null   float64
 6   ovr_rk_x       5100 non-null   float64
 7   grade_x        5767 non-null   float64
 8   text1          2992 non-null   object 
 9   text2          3848 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     12901 non-null  float64
 13  overall        11343 non-null  float64
 14  round          11343 non-null  float64
 15  team_abbr      11343 non-null  object 
dtypes: float64(7), int64(1), object(8)
memory usage: 1.7+ MB


In [8]:
texts = scout[(scout.text1.notna())|(scout.text2.notna())|(scout.text3.notna())|(scout.text4.notna())]

In [15]:
texts['report'] = None

C:\Users\capta\AppData\Local\Temp\ipykernel_11176\982957188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts['report'] = None


In [36]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4883 entries, 7375 to 12904
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  4883 non-null   object 
 1   pos_abbr_x     4883 non-null   object 
 2   weight_x       4883 non-null   int64  
 3   height_x       4883 non-null   float64
 4   school_x       4883 non-null   object 
 5   pos_rk_x       4763 non-null   float64
 6   ovr_rk_x       4165 non-null   float64
 7   grade_x        4769 non-null   float64
 8   text1          2992 non-null   object 
 9   text2          3848 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     4882 non-null   float64
 13  overall        4040 non-null   float64
 14  round          4040 non-null   float64
 15  team_abbr      4040 non-null   object 
 16  report         4883 non-null   object 
dtypes: float64(7), int64(1), object(9)
memory usage:

In [20]:
texts.loc[(texts.text1.notna())&(texts.text2.isna()),'report'] = texts.loc[(texts.text1.notna())&(texts.text2.isna())].text1 

In [26]:
texts.loc[(texts.text2.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text2.notna())&(texts.text1.isna())].text2 

In [29]:
texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna()),'report'] = texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna())].text3 

In [35]:
texts.loc[(texts.report.isna()),'report'] = texts.loc[(texts.report.isna())].text1

In [38]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4883 entries, 7375 to 12904
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  4883 non-null   object 
 1   pos_abbr_x     4883 non-null   object 
 2   weight_x       4883 non-null   int64  
 3   height_x       4883 non-null   float64
 4   school_x       4883 non-null   object 
 5   pos_rk_x       4763 non-null   float64
 6   ovr_rk_x       4165 non-null   float64
 7   grade_x        4769 non-null   float64
 8   text1          2992 non-null   object 
 9   text2          3848 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     4882 non-null   float64
 13  overall        4040 non-null   float64
 14  round          4040 non-null   float64
 15  team_abbr      4040 non-null   object 
 16  report         4883 non-null   object 
dtypes: float64(7), int64(1), object(9)
memory usage:

In [44]:
texts.report.head(1)

7375    Nephew of Michigan State head coach John L. Sm...
Name: report, dtype: object

In [46]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.stem import snowball

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\capta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
cv = CountVectorizer()

In [ ]:
texts.to_parquet()